<a href="https://colab.research.google.com/github/hyun-hyang/ResumEmphasizer/blob/main/resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 1.1 MB 55.7 MB/s 
     |████████████████████████████████| 4.4 MB 57.1 MB/s 
     |████████████████████████████████| 101 kB 8.8 MB/s 
     |████████████████████████████████| 6.6 MB 54.5 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy
from spacy.tokens import DocBin
import json

In [ ]:
spacy.__version__

'3.4.1'

In [ ]:
!nvidia-smi

Wed Aug 10 14:58:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |    312MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone 'https://github.com/hyun-hyang/ResumEmphasizer.git'

Cloning into 'ResumEmphasizer'...
remote: Enumerating objects: 7491, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 7491 (delta 35), reused 121 (delta 35), pack-reused 7368
Receiving objects: 100% (7491/7491), 38.16 MiB | 13.35 MiB/s, done.
Resolving deltas: 100% (805/805), done.


In [ ]:
%cd /content/ResumEmphasizer
!git pull

/content/ResumEmphasizer
Already up to date.


In [ ]:
cv_data = json.load(open('/content/ResumEmphasizer/Data_annoctated/merged res.json','r'))

In [ ]:
len(cv_data)

99

In [ ]:
!python -m spacy init fill-config /content/ResumEmphasizer/Data/base_config.cfg /content/ResumEmphasizer/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/ResumEmphasizer/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [27]:
cv_data[0]

{'id': 297,
 'label': [[0, 13, 'ability'],
  [623, 655, 'work experience'],
  [671, 724, 'work experience'],
  [1202, 1249, 'work experience'],
  [1274, 1349, 'work experience'],
  [1399, 1437, 'work experience'],
  [1600, 1655, 'award'],
  [1686, 1712, 'major'],
  [1713, 1716, 'degree'],
  [1725, 1757, 'college'],
  [1900, 1921, 'education'],
  [2020, 2026, 'skill'],
  [2027, 2034, 'skill'],
  [2035, 2041, 'skill'],
  [2058, 2074, 'skill'],
  [2075, 2095, 'skill'],
  [2097, 2100, 'skill'],
  [2101, 2106, 'skill'],
  [2108, 2112, 'skill'],
  [2113, 2123, 'skill'],
  [2125, 2129, 'skill'],
  [2130, 2133, 'skill'],
  [2135, 2138, 'skill'],
  [2165, 2182, 'ability'],
  [2183, 2198, 'ability'],
  [2199, 2215, 'ability'],
  [2216, 2229, 'ability'],
  [2336, 2363, 'skill'],
  [2402, 2420, 'skill'],
  [2456, 2477, 'skill']],
 'text': 'Web Developer Resume Template (Full Text Version)\nOlivier Baudet\nAddress: 44 place de Miremont, 77170, Paris, France\nEmail address: olivierbaudet@gmail.com\n

In [ ]:
def get_spacy_doc(file, data):
  nlp=spacy.blank('en')
  db=DocBin()

  for item in data:
    text = item['text']
    doc=nlp.make_doc(text)
    annot=item['label']

    
    ents=[]
    entity_indices=[]

    for start, end, label in annot:
      skip_entity=False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
         continue
        
      entity_indices = entity_indices + list(range(start, end))

      try:
        span=doc.char_span(start,end,label=label,alignment_mode='contract')
      except:
        continue
        
      if span is None:
        err_data=str([start,end]) + "     " + str(text) + "\n"
        file.write(err_data)
        
      else:
        ents.append(span)

    try:
        doc.ents=ents
        db.add(doc)
    except:
      pass


  return db

In [ ]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(cv_data,test_size=0.3)

In [ ]:
len(train), len(test)

(69, 30)

In [ ]:
file=open('error.txt','w')

db=get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

In [ ]:
len(db.tokens)

30

In [ ]:
!python -m spacy train /content/ResumEmphasizer/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-10 14:59:11,002] [INFO] Set up nlp object from config
[2022-08-10 14:59:11,013] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-10 14:59:11,017] [INFO] Created vocabulary
[2022-08-10 14:59:11,018] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 486kB/s]
Downloading: 100% 878k/878k [00:01<00:00, 655kB/s]
Downloading: 100% 446k/446k [00:01<00:00, 414kB/s]
Downloading: 100% 1.29M/1.29M [00:01<00:00, 982kB/s]
Downloading: 100% 478M/478M [00:06<00:00, 77.1MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model 

Model Test

In [ ]:
nlp = spacy.load('/content/ResumEmphasizer/output/model-best')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!zip -r /content/ResumEmphasizer/best.zip /content/ResumEmphasizer/output/model-best/


updating: content/ResumEmphasizer/output/model-best/ (stored 0%)
updating: content/ResumEmphasizer/output/model-best/ner/ (stored 0%)
updating: content/ResumEmphasizer/output/model-best/ner/moves (deflated 71%)
updating: content/ResumEmphasizer/output/model-best/ner/model (deflated 8%)
updating: content/ResumEmphasizer/output/model-best/ner/cfg (deflated 33%)
updating: content/ResumEmphasizer/output/model-best/tokenizer (deflated 81%)
updating: content/ResumEmphasizer/output/model-best/config.cfg (deflated 61%)
updating: content/ResumEmphasizer/output/model-best/transformer/ (stored 0%)
updating: content/ResumEmphasizer/output/model-best/transformer/model (deflated 14%)
updating: content/ResumEmphasizer/output/model-best/transformer/cfg (stored 0%)
updating: content/ResumEmphasizer/output/model-best/vocab/ (stored 0%)
updating: content/ResumEmphasizer/output/model-best/vocab/strings.json (deflated 72%)
updating: content/ResumEmphasizer/output/model-best/vocab/vectors.cfg (stored 0%)
up

In [ ]:
from google.colab import files
files.download("/content/ResumEmphasizer/best.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 3.0 MB/s 


In [ ]:
import sys, fitz

In [ ]:
for x in range(1,11):
  fname='/content/ResumEmphasizer/test_cv/'+str(x)+'.pdf'
  doc=fitz.open(fname)
  print('resume num '+str(x)+' test')
  text=" "
  for page in doc:
    text=text+str(page.get_text())
  text=text.strip()
  text=' '.join(text.split())
  
  doc=nlp(text)
  for ent in doc.ents:
    print(ent.text, " ->>>>>> ", ent.label_)
  print('\n')

resume num 1 test


resume num 2 test
• 1 year and 3 months experience in web application development using  ->>>>>>  work experience
Java  ->>>>>>  skill
J2EE  ->>>>>>  skill
• Expertise in  ->>>>>>  work experience
JSP  ->>>>>>  skill
• Working experience  ->>>>>>  work experience
• Good experience  ->>>>>>  work experience
• Expertise in  ->>>>>>  work experience
HTML  ->>>>>>  skill
Javascript  ->>>>>>  skill
• Good Knowledge  ->>>>>>  work experience
• Aware of  ->>>>>>  work experience
• Hands on  ->>>>>>  work experience
• Requirement Analysis  ->>>>>>  work experience
Java Developer Tata Consultancy Services (TCS)  ->>>>>>  work experience
• Title:  ->>>>>>  work experience
• Client:  ->>>>>>  work experience
• Project Module  ->>>>>>  work experience
• Role:  ->>>>>>  work experience
Java developer  ->>>>>>  ability
• Technologies used  ->>>>>>  work experience
Core Java  ->>>>>>  skill
JSP  ->>>>>>  skill
Spring  ->>>>>>  skill
JDBC  ->>>>>>  skill
Webservices  ->>>>>>  skill